These functions are based on those from the Luna16 tutorial and have been modified for my needs. https://luna16.grand-challenge.org/Tutorial/


In [1]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
from random import randint
import csv
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import time

We define now a function to:

- Open the image 
- Store it into a numpy array
- Extract the following info: Pixel Spacing, Origin

This function takes as input the name of the image and returns:

- The array corresponding to the image (numpyImage)
- Origin (numpyOrigin)
- PixelSpacing (numpySpacing)

In [2]:
def load_itk_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
     
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
     
    return numpyImage, numpyOrigin, numpySpacing

Since the coordinates of the candidates are given in World Coordinates, we now need to transform from world coordinates to voxel coordinates. We define now a function to do that. Please note that the transformation below is only valid if there is no rotation component in the transformation matrix. For all CT images in our dataset, there is no rotation component so that this formula can be used. This function takes as inputs:

- The world coordinates
- The origin
- The pixel Spacing

This function returns:

- Voxel coordinates (voxelCoord)

In [3]:
def worldToVoxelCoord(worldCoord, origin, spacing):
     
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord

We want to extract now some features from the candidates. We define some normalized planes to extract views from the candidates


In [4]:
def normalizePlanes(npzarray):
     
    maxHU = 400.
    minHU = -1000.
 
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray[npzarray>1] = 1.
    npzarray[npzarray<0] = 0.
    return npzarray

After having defined these auxiliary functions, we can now define the main part of our script. First we:

- Specificy the path where the file with the list of candidates is (cand_path)

In [2]:
# Set data split to 'train', 'test', or 'validation'

data_split = 'validation'

In [6]:
cand_path = 'annotations.csv'

In [7]:
# load candidates

cands = pd.read_csv(cand_path)

In [3]:
# get list of the images saved to disk

ids = os.listdir(f'{data_split}_data_raw')
print(f'{len(ids)} files in directory')

354 files in directory


In [9]:
# drop filetype extension

for i, val in enumerate(ids):
    ids[i] = val[:-4]

ids = list(pd.Series(ids).drop_duplicates())

Since I will not be using the entire dataset, I need to trim down the candidate list to only those values which I have saved on my disk. I will create a new df with only these values.

In [10]:
cands_trim = pd.DataFrame(columns=cands.columns)

for i, val in enumerate(ids):
    cands_trim = cands_trim.append(cands.loc[cands.seriesuid == ids[i], :], ignore_index=True)

Loop through list of trimmed candidates, load respective image file, extract patch of lung nodule, save file.

In [11]:
start_time = time.time()

for i in range(cands_trim.shape[0]):
    
    cand = cands_trim.loc[i,:]
    voxelWidth = 65
    
    # Use a random offset to allow the model to identify the X, Y location
    
    offset_y = randint(-10, 10)
    offset_x = randint(-10, 10)
    
    # load image
    numpyImage, numpyOrigin, numpySpacing = load_itk_image(f'{data_split}_data_raw/{cand.seriesuid}.mhd')
    if numpyImage.shape:
        print(f'Image loaded {i}')
    
    # Create and save patch of lung nodule
    worldCoord = np.asarray([float(cand.coordZ),float(cand.coordY),float(cand.coordX)])
    voxelCoord = worldToVoxelCoord(worldCoord, numpyOrigin, numpySpacing)
    
    z = voxelCoord[0]
    y = voxelCoord[1] + offset_y
    x = voxelCoord[2] + offset_x
    
    try:
        patch = numpyImage[int(z),
                           int(y-voxelWidth/2):int(y+voxelWidth/2),
                           int(x-voxelWidth/2):int(x+voxelWidth/2)]
        
        if patch.sum() != 0:
            patch = normalizePlanes(patch)
            Image.fromarray(patch*255).convert('L').save(f'{data_split}/pos/pos_{i}.tiff')
            
            cands_trim.loc[i, 'voxel_x'] = voxelWidth/2 - offset_x
            cands_trim.loc[i, 'voxel_y'] = voxelWidth/2 - offset_y
            cands_trim.loc[i, 'voxel_z'] = z
        else:
            print(f'A null patch was found during iteration {i}')
            
    except:
        print(f'Error creating positive tile at iteration {i}')

print(f'That took {time.time()-start_time:.2f} seconds to complete')

Image loaded 0
Image loaded 1
Image loaded 2
Image loaded 3
Image loaded 4
Image loaded 5
Image loaded 6
Image loaded 7
Image loaded 8
Image loaded 9
Image loaded 10
Image loaded 11
Image loaded 12
Image loaded 13
Image loaded 14
Image loaded 15
Image loaded 16
Image loaded 17
Image loaded 18
Image loaded 19
Image loaded 20
Image loaded 21
Image loaded 22
Image loaded 23
Image loaded 24
Image loaded 25
A null patch was found during iteration 25
Image loaded 26
A null patch was found during iteration 26
Image loaded 27
Image loaded 28
Image loaded 29
Image loaded 30
Image loaded 31
Image loaded 32
Image loaded 33
Image loaded 34
Image loaded 35
Image loaded 36
Image loaded 37
Image loaded 38
Image loaded 39
Image loaded 40
Image loaded 41
Image loaded 42
Image loaded 43
Image loaded 44
Image loaded 45
Image loaded 46
Image loaded 47
Image loaded 48
Image loaded 49
Image loaded 50
Image loaded 51
Image loaded 52
Image loaded 53
Image loaded 54
Image loaded 55
Image loaded 56
Image loaded

Save trimmed candidates list for modelling. 

In [12]:
cands_trim.to_csv(f'candidates_{data_split}.csv')

Now we will create the negative patches, to increase the probability that our negative patches do not have a nodule in them we will only take patches from series that only contain 1 nodule and modify the z coordinate. We will take a number of patches from each series based on the number of patches needed to keep a 1:1 positive to negative ratio.

In [13]:
single_nods = cands_trim.groupby('seriesuid').size().sort_values()
single_nods = single_nods[single_nods == 1]
n_per_series = int(cands_trim.shape[0]/len(single_nods))

In [14]:
print(n_per_series)

4


In [15]:
start_time = time.time()

count = 0

for i, seriesuid in enumerate(single_nods.index):
    
    cand = cands_trim.loc[cands_trim.seriesuid == seriesuid]
    
    # load image
    numpyImage, numpyOrigin, numpySpacing = load_itk_image(f'{data_split}_data_raw/{seriesuid}.mhd')
    if numpyImage.shape:
        print(f'Image loaded {i}')
        
    worldCoord = np.asarray([float(cand.coordZ),float(cand.coordY),float(cand.coordX)])
    voxelCoord = worldToVoxelCoord(worldCoord, numpyOrigin, numpySpacing)
    
    z = voxelCoord[0]
    y = voxelCoord[1] + offset_y
    x = voxelCoord[2] + offset_x
        
    for j in range(n_per_series):
        
        if z < 100:
            z = z + 15*(j+1)
        else:
            z = z - 15*(j+1)
    
        try:
            patch = numpyImage[int(z),
                               int(y-voxelWidth/2):int(y+voxelWidth/2),
                               int(x-voxelWidth/2):int(x+voxelWidth/2)]

            if patch.sum() != 0:
                patch = normalizePlanes(patch)
                Image.fromarray(patch*255).convert('L').save(f'{data_split}/neg/neg_{count}.tiff')
                count = count + 1

            else:
                print(f'A null patch was found during iteration {i}')

        except:
            print(f'Error creating negative tile at iteration {i}')
    
print(f'That took {time.time()-start_time:.2f} seconds to complete')

Image loaded 0
Image loaded 1
Image loaded 2
Image loaded 3
Image loaded 4
Image loaded 5
Image loaded 6
Image loaded 7
Image loaded 8
Image loaded 9
Image loaded 10
Image loaded 11
Image loaded 12
Image loaded 13
Image loaded 14
Image loaded 15
Image loaded 16
Image loaded 17
Image loaded 18
Image loaded 19
Image loaded 20
Image loaded 21
Image loaded 22
Image loaded 23
Error creating negative tile at iteration 23
Image loaded 24
Image loaded 25
Error creating negative tile at iteration 25
Image loaded 26
Image loaded 27
Image loaded 28
A null patch was found during iteration 28
A null patch was found during iteration 28
A null patch was found during iteration 28
A null patch was found during iteration 28
Image loaded 29
Image loaded 30
Image loaded 31
Image loaded 32
Image loaded 33
A null patch was found during iteration 33
A null patch was found during iteration 33
A null patch was found during iteration 33
Error creating negative tile at iteration 33
Image loaded 34
Image loaded 3